# MCMC3.5: Jackknife Resampling

To estimate the error of observables accurately, it is recommended to use the Jackknife resampling method.

## Binning

In [1]:
using ResumableFunctions
using SparseArrays
using LinearAlgebra
const Jx = 1 / 3 # oppposite sign to Motome's
const Jy = 1 / 3
const Jz = 1 / 3
"""
Metropolis method
"""
function Metropolis(βF::Float64, βFnew::Float64)::Bool
    βF - βFnew > log(rand())
end
"""
Generating a honeycomb lattice with an open boundary condition.
"""
function openhoneycomb(Lx::Int64, Ly::Int64)::Tuple
    N = 2Lx * Ly
    nnx = zip(1 : 2 : (N - 1), 2 : 2 : N)
    nny = Iterators.flatten((zip((1 + 2i) : 2Lx : (2Lx * (Ly - 1)  + 1 + 2i), 2i : 2Lx : (2Lx * (Ly - 1)  + 2i)) for i in 1 : (Lx - 1)))
    nnz = zip(1 : 2 : (N - 1), Iterators.flatten(((2Lx + 2) : 2 : N, 2 : 2 : 2Lx)))
    plaquette = Iterators.flatten(zip((Lx * (i - 1) + 1) : (Lx * (i - 1) + Lx - 1), (Lx * (i - 1) + 2) : (Lx * (i - 1) + Lx)) for i in 1 : Ly)
    N, nnx, nny, nnz, plaquette
end
@resumable function measurementflux(method::Function, lattice::Function, β::Float64, Lx::Int64, Ly::Int64)::Float64
    N, nnx, nny, nnz, plaquette = lattice(Lx, Ly)
    iter = Iterators.flatten((Iterators.product(J, nn) for (J, nn) in [(Jx, nnx), (Jy, nny), (Jz, nnz)]))
    h = spzeros(Complex{Float64}, N, N)
    for (J, nn) in iter
        h[nn[1], nn[2]] = 2.0im * J
        h[nn[2], nn[1]] = -2.0im * J
    end
    NNz = collect(nnz)
    Nz = length(NNz)
    η = ones(Int64, Nz)
    βF = 0.0
    hdense = Array(h)
    plaq = collect(plaquette)
    Np = length(plaq)
    while true
        for i in 1 : Nz
            j = rand(1 : Nz)
            hdense[NNz[j][1], NNz[j][2]] = -hdense[NNz[j][1], NNz[j][2]]
            hdense[NNz[j][2], NNz[j][1]] = -hdense[NNz[j][2], NNz[j][1]]
            ev = eigvals(Hermitian(hdense))
            positiveev = Iterators.drop(ev, N >> 1)
            βFnew = -sum((log(2.0 * cosh(β * ϵ / 2.0)) for ϵ in positiveev))
            if method(βF, βFnew)
                η[j] = -η[j]
                βF = βFnew
            else
                hdense[NNz[j][1], NNz[j][2]] = -hdense[NNz[j][1], NNz[j][2]]
                hdense[NNz[j][2], NNz[j][1]] = -hdense[NNz[j][2], NNz[j][1]]
            end
        end
        @yield sum((η[k] * η[l] for (k, l) in plaq)) / Np
    end
end

measurementflux (generic function with 1 method)

One reason of the error bar is the discreteness of the returned value `flux`.

In [2]:
mcstep = Iterators.drop(measurementflux(Metropolis, openhoneycomb, 25.0, 4, 4), 10000)
foreach(println, Iterators.take(mcstep, 10))

0.0
0.16666666666666666
0.3333333333333333
-0.16666666666666666
0.16666666666666666
0.3333333333333333
0.6666666666666666
0.0
0.6666666666666666
0.3333333333333333


We have to flatten these quantized values by binning.

In [3]:
using Statistics
Nsample = 10000
Nbin = 10
Nbinsize = Nsample ÷ Nbin
iter = Iterators.partition(Iterators.take(mcstep, Nsample), Nbinsize)
bin = collect(map(mean, iter))

10-element Array{Float64,1}:
 0.29933333333333334
 0.304              
 0.2976666666666667 
 0.3225             
 0.2965             
 0.31016666666666665
 0.30383333333333323
 0.30650000000000005
 0.2854999999999999 
 0.2963333333333334 

Now it works!

In [4]:
m = mean(bin)
s = stdm(bin, m) / sqrt(length(bin))
println("$m ± $s")

0.30223333333333335 ± 0.0031171766269264905


`Nbinsize` has to be determined based on the autocorrelation. In order to reduce `Nbin` to get a more acculate result at low temperature, we need to implement some global updating algorithm.

## Delete-1 jackknife resampling

Delete-1 jackknife resampling is simply implemented in https://github.com/ararslan/Jackknife.jl. However, the function is limited, so I will newly define functions for the jackknife resampling.

In [5]:
"""
Delete one sample from given samples. before and after are applied before and after taking an expectation value.
"""
function leaveoneout(before::Function, after::Function, v::AbstractVector)
    ind = eachindex(v)
    map(i -> after(mean(map(before, view(v, filter(!isequal(i), ind))))), ind)
end
"""
Calculate an expectation value by jackknife.
"""
meanJ(b::Function, a::Function, v::AbstractVector) = mean(leaveoneout(b, a, v))
"""
Calculate an error bar with given mean by jackknife.
"""
stdmJ(b::Function, a::Function, v::AbstractVector, m) = stdm(leaveoneout(b, a, v), m, corrected = false) * sqrt(length(v) - 1)
"""
Calculate an error bar by jackknife.
"""
stdJ(b::Function, a::Function, v::AbstractVector) = stdmJ(b, a, v, meanJ(b, a, v))

stdJ

The functions are based on Statistics.jl and Jackknife.jl, so please see their reference to know how it works. I again use `measurementEf` as a demonstration.

In [6]:
@resumable function measurementEf(method::Function, lattice::Function, β::Float64, Lx::Int64, Ly::Int64)::Vector{Float64}
    N, nnx, nny, nnz, plaquette = lattice(Lx, Ly)
    iter = Iterators.flatten((Iterators.product(J, nn) for (J, nn) in [(Jx, nnx), (Jy, nny), (Jz, nnz)]))
    h = spzeros(Complex{Float64}, N, N)
    for (J, nn) in iter
        h[nn[1], nn[2]] = 2.0im * J
        h[nn[2], nn[1]] = -2.0im * J
    end
    NNz = collect(nnz)
    Nz = length(NNz)
    η = ones(Int64, Nz)
    βF = 0.0
    β₂ = β * 0.5
    hdense = Array(h)
    ev = zeros(Float64, N)
    while true
        for i in 1 : Nz
            j = rand(1 : Nz)
            hdense[NNz[j][1], NNz[j][2]] = -hdense[NNz[j][1], NNz[j][2]]
            hdense[NNz[j][2], NNz[j][1]] = -hdense[NNz[j][2], NNz[j][1]]
            evnew = eigvals(Hermitian(hdense))
            βFnew = -sum(@. log(exp(β₂ * evnew[(N >> 1 + 1) : end]) + exp(-β₂ * evnew[(N >> 1 + 1) : end])))
            if method(βF, βFnew)
                η[j] = -η[j]
                βF = βFnew
                ev .= evnew
            else
                hdense[NNz[j][1], NNz[j][2]] = -hdense[NNz[j][1], NNz[j][2]]
                hdense[NNz[j][2], NNz[j][1]] = -hdense[NNz[j][2], NNz[j][1]]
            end
        end
        Ef = -sum(@. ev[(N >> 1 + 1) : end] * tanh(β₂ * ev[(N >> 1 + 1) : end] )) * 0.5
        ∂Ef∂β = -sum(@. (ev[(N >> 1 + 1) : end] ^ 2) * (sech(β₂ * ev[(N >> 1 + 1) : end]) ^ 2)) * 0.25
        @yield [Ef, ∂Ef∂β]
    end
end

measurementEf (generic function with 1 method)

It is ok to first assume the bin size to be 1.

In [7]:
const β = 1.0
mcstep2 = Iterators.drop(measurementEf(Metropolis, openhoneycomb, β, 4, 4), 10000)
iter2 = Iterators.take(mcstep2, Nsample)
data = collect(iter2)

10000-element Array{Array{Float64,1},1}:
 [-4.20917, -3.10421]
 [-4.20917, -3.1042] 
 [-4.20683, -3.09509]
 [-4.20799, -3.09958]
 [-4.21161, -3.11383]
 [-4.20684, -3.09513]
 [-4.20658, -3.09366]
 [-4.20927, -3.10472]
 [-4.21022, -3.10809]
 [-4.208, -3.09965]  
 [-4.21161, -3.11383]
 [-4.2103, -3.10847] 
 [-4.21146, -3.11294]
 ⋮                   
 [-4.20426, -3.08468]
 [-4.21162, -3.11388]
 [-4.21162, -3.11391]
 [-4.21394, -3.12285]
 [-4.20914, -3.10396]
 [-4.2103, -3.10846] 
 [-4.21023, -3.1081] 
 [-4.20786, -3.09883]
 [-4.208, -3.09962]  
 [-4.21146, -3.11291]
 [-4.20674, -3.09462]
 [-4.20684, -3.09518]

By setting `after` = `before` = `identity`, `meanJ` and `stdmJ `work in the same way as `mean` and `stdm`, respectively.

In [8]:
m2 = meanJ(identity, identity, data)
s2 = stdmJ(identity, identity, data, m2)
println("Ef = $(m2[1]) ± $(s2[1]), ∂Ef∂β = $(m2[2]) ± $(s2[2])")

Ef = -4.209140126447901 ± 2.068936259148181e-5, ∂Ef∂β = -3.1040071041504516 ± 8.133661144884049e-5


This agrees with the standard estimation method for the error bars.

In [9]:
std(data) / sqrt(length(data))

2-element Array{Float64,1}:
 2.0689362590090705e-5
 8.13366114484194e-5  

For such mean values, the jackknife resampling is apparently overkill. However, to estimate the error for the values like the specific heat, the jackknife resampling is very effective.

In [10]:
"""
Function applied before mean to calculate heat capacity.
"""
TTCv(v::Vector{Float64}) = [v[1] ^ 2 - v[2], v[1]]
"""
Function applied after mean to calculate heat capacity.
"""
function Cv(β::Float64)::Function
    β² = β ^ 2
    meanTTCv::Vector{Float64} -> β² * (meanTTCv[1] - meanTTCv[2] ^ 2)
end
m3 = meanJ(TTCv, Cv(β), data)
s3 = stdmJ(TTCv, Cv(β), data, m3)
println("Cv = $m3 ± $s3")

Cv = 3.104011384219618 ± 8.133668254212314e-5


## Autocorrelation

The simplest way to estimate autocorrelation is by changing the size of binning and estimating its errors by jackknife resampling.

In [13]:
binning = Iterators.partition(data, 2)
bin2 = mean.(TTCv, binning)

5000-element Array{Array{Float64,1},1}:
 [20.8213, -4.20917]
 [20.7997, -4.20741]
 [20.8221, -4.20923]
 [20.8058, -4.20793]
 [20.8205, -4.20911]
 [20.8433, -4.21096]
 [20.8343, -4.21024]
 [20.7981, -4.2073] 
 [20.8371, -4.21044]
 [20.813, -4.20851] 
 [20.82, -4.20908]  
 [20.806, -4.20793] 
 [20.8578, -4.21213]
 ⋮                  
 [20.8215, -4.20919]
 [20.8428, -4.21092]
 [20.8129, -4.2085] 
 [20.8428, -4.21092]
 [20.8368, -4.21042]
 [20.8147, -4.20863]
 [20.806, -4.20794] 
 [20.8659, -4.21278]
 [20.828, -4.20972] 
 [20.8195, -4.20904]
 [20.8281, -4.20973]
 [20.792, -4.20679] 

In [14]:
m4 = meanJ(identity, Cv(β), bin2)
s4 = stdmJ(identity, Cv(β), bin2, m4)
println("Cv = $m4 ± $s4")

Cv = 3.1040113842195596 ± 8.21955075233793e-5


The fact that the binsize does not affect the expectation value (or the errorbar) too much means that the autocorrelation length is about 1 step. Note that at low temperature the binsize strongly affects the expectation value, which means that the binsize must be taken to be large enough. Here I estimate the autocorrelation length by another method.

In [15]:
using StatsBase
StatsBase.autocor(first.(data))

41-element Array{Float64,1}:
  1.0                   
  0.02286908196515771   
 -0.018888984422812474  
  0.021829454619408578  
  0.016549904949626704  
 -0.0155068840216553    
 -0.013197080384697673  
 -0.011556367455848426  
 -0.003343296294953272  
  0.0009544150946894916 
  0.0028004311822582384 
  0.00301628991521423   
  0.015587019877696114  
  ⋮                     
 -0.011264603693869172  
 -0.010329078420158462  
 -0.010026730979033106  
  0.006263246948410155  
  0.0098330230261655    
 -0.004421142020693845  
 -0.00616530035362706   
  0.021025845624635647  
  0.010756160173305423  
  0.0012214071400869463 
 -0.003630758604304952  
 -0.00026073361755733396

Rapid decay in the autocorrelation function means that the autocorrelation length is less than 1.

## Bootstrap method

~ under construction ~

**Exercise**: implement a Bootstrap method.